In [1]:
import tensorflow as tf
import json
import cv2 as cv
import numpy as np

In [2]:
# Reading data from .json

In [3]:
def readNameImages(filename, percent):
    
    read       = json.loads(open(filename).read())
    cant_Train = int(len(read)*percent)
    cant_Test  = len(read)-cant_Train
    
    
    qid        = []
    image_name = []
    question   = []
    answer     = [] 
    
    i = 0
    for _ in range(cant_Train):
        qid.append(read[i]['qid'])
        image_name.append(read[i]['image_name'])
        question.append(   str(read[i]['question'])    )
        answer.append(   str(read[i]['answer'])   )
        
        i += 1
        
    qid_Test        = []
    image_name_Test = []
    question_Test   = []
    answer_Test     = [] 
    
    for _ in range(cant_Test):
        qid_Test.append(read[i]['qid'])
        image_name_Test.append(read[i]['image_name'])
        question_Test.append(   str(read[i]['question'])    )
        answer_Test.append(   str(read[i]['answer'])   )
        
        i += 1
        
    return qid, image_name, question, answer, qid_Test, image_name_Test, question_Test, answer_Test
    

In [4]:
percent_Train = 0.9
qid, image_name, question, answer, qid_Test, image_name_Test, question_Test, answer_Test = readNameImages('data.json', percent_Train)
print (len(question))
print (len(question_Test))

2023
225


## Question Answering Module

In [5]:
import json 
import os
import collections
import tensorflow as tf 
import re
import h5py
import argparse
import sys 
import numpy as np 
import pandas as pd
import pickle

In [6]:
FLAGS = None
BUFFER_TOKENS = ['<NULL>', '<START>', '<END>', '<UNK>']

In [7]:
def _parse_sentence(s):
    s = s.replace('.', '')
    s = s.replace(',', '')
    s = s.replace('"', '')
    s = s.replace("'", '')
    s = s.replace("?", '')
    s = s.lower()
    s = re.sub("\s\s+", " ", s)
    s = s.split(' ')
    return s

In [8]:
def getData(qid, image_name, question, answer):
    data = {'SerialNo': qid,
         'ImageNo': image_name,
         'Question': question,
         'Answer': answer}
    df = pd.DataFrame.from_dict(data)
    
    return df

In [9]:
df_Train = getData(qid, image_name, question, answer)
df_Train.head()

,SerialNo,ImageNo,Question,Answer
0,0,synpic54610.jpg,Are regions of the brain infarcted?,Yes
1,1,synpic29265.jpg,Are the lungs normal appearing?,No
2,2,synpic29265.jpg,Is there evidence of a pneumothorax,No
3,3,synpic28602.jpg,What type of imaging does this not represent?,ultrasound
4,4,synpic29265.jpg,Is this a MRI of the chest?,no


In [10]:
df_Test = getData(qid_Test, image_name_Test, question_Test, answer_Test)
df_Test[0:10]

,SerialNo,ImageNo,Question,Answer
0,2023,synpic20260.jpg,What is located near the Left kidney,Splenule
1,2024,synpic18461.jpg,Is the right costophrenic angle sharp?,no
2,2025,synpic18461.jpg,Is the right costophrenic angle easily visuali...,no
3,2026,synpic55948.jpg,Are the ventricles symmetrical?,No
4,2027,synpic55948.jpg,Are the ventricles the same size?,No
5,2028,synpic34836.jpg,What does the brighter outer ring of the lesio...,necrosis
6,2029,synpic34836.jpg,What is indicated by the bright ring around th...,necrosis
7,2030,synpic27601.jpg,Where in relation to the mass is the left kidn...,posteriorly
8,2031,synpic27601.jpg,Where is the mass located in relation to the l...,posteriorly
9,2032,synpic19782.jpg,On which side is air under the diaphragm?,Right


In [150]:
def _create_init_dic(df):
   
    bool_mat = []
    
    for i in range(0,len(df)):
        bool_mat.append(df['Question'].iloc[i][len(df['Question'].iloc[i])-1]=='?')
        
    df = df[bool_mat]
    df['Q_parsed'] = [ _parse_sentence(s) for s in df['Question']]
    df['A_parsed'] = [ _parse_sentence(s) for s in df['Answer']]
    
    return df


#Training Dataset
dic_df=_create_init_dic(df_Train)

#Test Dataset
dic_v_df=_create_init_dic(df_Test)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [151]:
print(len(dic_df))
dic_df.head()

1952


,SerialNo,ImageNo,Question,Answer,Q_parsed,A_parsed
0,0,synpic54610.jpg,Are regions of the brain infarcted?,Yes,"[are, regions, of, the, brain, infarcted]",[yes]
1,1,synpic29265.jpg,Are the lungs normal appearing?,No,"[are, the, lungs, normal, appearing]",[no]
3,3,synpic28602.jpg,What type of imaging does this not represent?,ultrasound,"[what, type, of, imaging, does, this, not, rep...",[ultrasound]
4,4,synpic29265.jpg,Is this a MRI of the chest?,no,"[is, this, a, mri, of, the, chest]",[no]
5,5,synpic28602.jpg,What is not pictured in this image?,The extremities,"[what, is, not, pictured, in, this, image]","[the, extremities]"


In [152]:
print(len(dic_v_df))
dic_v_df.head()

213


,SerialNo,ImageNo,Question,Answer,Q_parsed,A_parsed
1,2024,synpic18461.jpg,Is the right costophrenic angle sharp?,no,"[is, the, right, costophrenic, angle, sharp]",[no]
2,2025,synpic18461.jpg,Is the right costophrenic angle easily visuali...,no,"[is, the, right, costophrenic, angle, easily, ...",[no]
3,2026,synpic55948.jpg,Are the ventricles symmetrical?,No,"[are, the, ventricles, symmetrical]",[no]
4,2027,synpic55948.jpg,Are the ventricles the same size?,No,"[are, the, ventricles, the, same, size]",[no]
5,2028,synpic34836.jpg,What does the brighter outer ring of the lesio...,necrosis,"[what, does, the, brighter, outer, ring, of, t...",[necrosis]


In [153]:
list_of_all_words = []
for i in range(0,len(dic_df)):
    list_of_all_words = list_of_all_words + dic_df['A_parsed'].iloc[i]+dic_df['Q_parsed'].iloc[i]

In [154]:
counter = collections.Counter(list_of_all_words)
TOTAL_VOCAB =len(counter)
vocab = counter.most_common(TOTAL_VOCAB)

In [155]:
## create word_to_idx, and idx_to_word
vocab = [i[0] for i in vocab]
word_to_idx = {}
idx_to_word = {}

# add in BUFFER_TOKENS
for i in range(len(BUFFER_TOKENS)):
    idx_to_word[int(i)] = BUFFER_TOKENS[i]
    word_to_idx[BUFFER_TOKENS[i]] = i

for i in range(len(vocab)):
    word_to_idx[vocab[i]] = i + len(BUFFER_TOKENS)
    idx_to_word[int(i + len(BUFFER_TOKENS))] = vocab[i]

In [156]:
pickle.dump(word_to_idx, open('word_to_idx.pkl', 'wb') )
pickle.dump(idx_to_word, open('idx_to_word.pkl', 'wb') )

In [157]:
PADDING_LEN = 17

def _convert_sentence_to_numbers(s):
    """Convert a sentence s (a list of words) to list of numbers using word_to_idx"""
    UNK_IDX = BUFFER_TOKENS.index('<UNK>')
    NULL_IDX = BUFFER_TOKENS.index('<NULL>')
    END_IDX = BUFFER_TOKENS.index('<END>')
    STR_IDX = BUFFER_TOKENS.index('<START>')
    s_encoded = [word_to_idx.get(w, UNK_IDX) for w in s]
    s_encoded = [STR_IDX] + s_encoded
    s_encoded += [END_IDX]
    s_encoded += [NULL_IDX] * (PADDING_LEN - 1 - len(s_encoded))
    return s_encoded

In [158]:
df_final = dic_df
all_answers = [_convert_sentence_to_numbers(s) for s in np.array(df_final['A_parsed'])] # list of numbers 
valid_rows1 = [i for i in range(len(all_answers)) if len(all_answers[i]) == PADDING_LEN-1]
df_final = df_final.iloc[valid_rows1,:]
df_final['A_Encoded']=[row for row in all_answers if len(row) == PADDING_LEN-1]
all_questions = [_convert_sentence_to_numbers(s) for s in np.array(df_final['Q_parsed'])] 
valid_rows2 = [i for i in range(len(all_questions)) if len(all_questions[i]) == PADDING_LEN-1]
df_final=df_final.iloc[valid_rows2,:]
df_final['Q_Encoded']=[row for row in all_questions if len(row) == PADDING_LEN-1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [159]:
df_final.head()

,SerialNo,ImageNo,Question,Answer,Q_parsed,A_parsed,A_Encoded,Q_Encoded
0,0,synpic54610.jpg,Are regions of the brain infarcted?,Yes,"[are, regions, of, the, brain, infarcted]",[yes],"[1, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 14, 802, 11, 4, 29, 468, 2, 0, 0, 0, 0, 0,..."
1,1,synpic29265.jpg,Are the lungs normal appearing?,No,"[are, the, lungs, normal, appearing]",[no],"[1, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 14, 4, 86, 24, 569, 2, 0, 0, 0, 0, 0, 0, 0..."
3,3,synpic28602.jpg,What type of imaging does this not represent?,ultrasound,"[what, type, of, imaging, does, this, not, rep...",[ultrasound],"[1, 803, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 10, 64, 11, 70, 28, 8, 231, 199, 2, 0, 0, ..."
4,4,synpic29265.jpg,Is this a MRI of the chest?,no,"[is, this, a, mri, of, the, chest]",[no],"[1, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 5, 8, 15, 45, 11, 4, 72, 2, 0, 0, 0, 0, 0,..."
5,5,synpic28602.jpg,What is not pictured in this image?,The extremities,"[what, is, not, pictured, in, this, image]","[the, extremities]","[1, 4, 804, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 10, 5, 231, 469, 9, 8, 13, 2, 0, 0, 0, 0, ..."


In [160]:
df_v_final=dic_v_df
all_answers = [_convert_sentence_to_numbers(s) for s in np.array(df_v_final['A_parsed'])] # list of numbers 
valid_rows3 = [i for i in range(len(all_answers)) if len(all_answers[i]) == PADDING_LEN-1]
df_v_final=df_v_final.iloc[valid_rows3,:]
df_v_final['A_Encoded']=[row for row in all_answers if len(row) == PADDING_LEN-1]
all_questions = [_convert_sentence_to_numbers(s) for s in np.array(df_v_final['Q_parsed'])] 
valid_rows4 = [i for i in range(len(all_questions)) if len(all_questions[i]) == PADDING_LEN-1]
df_v_final=df_v_final.iloc[valid_rows4,:]
df_v_final['Q_Encoded']=[row for row in all_questions if len(row) == PADDING_LEN-1]

In [161]:
df_v_final.head()

,SerialNo,ImageNo,Question,Answer,Q_parsed,A_parsed,A_Encoded,Q_Encoded
1,2024,synpic18461.jpg,Is the right costophrenic angle sharp?,no,"[is, the, right, costophrenic, angle, sharp]",[no],"[1, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 5, 4, 16, 154, 242, 862, 2, 0, 0, 0, 0, 0,..."
2,2025,synpic18461.jpg,Is the right costophrenic angle easily visuali...,no,"[is, the, right, costophrenic, angle, easily, ...",[no],"[1, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 5, 4, 16, 154, 242, 3, 75, 2, 0, 0, 0, 0, ..."
3,2026,synpic55948.jpg,Are the ventricles symmetrical?,No,"[are, the, ventricles, symmetrical]",[no],"[1, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 14, 4, 79, 857, 2, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,2027,synpic55948.jpg,Are the ventricles the same size?,No,"[are, the, ventricles, the, same, size]",[no],"[1, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 14, 4, 79, 4, 967, 67, 2, 0, 0, 0, 0, 0, 0..."
5,2028,synpic34836.jpg,What does the brighter outer ring of the lesio...,necrosis,"[what, does, the, brighter, outer, ring, of, t...",[necrosis],"[1, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 10, 28, 4, 3, 1149, 167, 11, 4, 20, 199, 2..."


In [162]:
#save preprocessed training data frame
df_final.to_pickle("train_df_final.pkl")
print( len(df_final) )

1933


In [163]:
#save preprocessed test data frame
df_v_final.to_pickle("test_df_v_final.pkl")
print( len(df_v_final) )

207


## Select the correct images

In [164]:
def readData(name):
    data = np.load(name)
    return data

In [165]:
features_Read = readData('features_images.npy')

In [166]:
cant_Train = int(  len(features_Read)*percent_Train )
cant_Test  = len(features_Read)-cant_Train
print(cant_Train)
print(cant_Test)

2023
225


In [167]:
features_Train = features_Read[0:cant_Train]
features_Test = features_Read[cant_Train:]
print(len(features_Train))
print(len(features_Test))

2023
225


In [168]:
#Take input as preprocssed images (features) and select valid rows based on above selection during training
features = np.array(features_Train)
features = features[valid_rows1,]
features = features[valid_rows2,]

In [169]:
print(len(features))

1933


In [170]:
pickle.dump(features, open('image_feature_train.pkl','wb'))

In [171]:
#Take input as preprocssed images (features) and select valid rows based on above selection during testing
features = np.array(features_Test)
features = features[valid_rows3,]
features = features[valid_rows4,]

In [172]:
print(len(features))

207


In [173]:
pickle.dump(features, open('image_feature_test.pkl','wb'))